In [1]:
import os
import numpy as np
import matplotlib.pyplot as plt
from tensorflow.keras.preprocessing.image import ImageDataGenerator, load_img, img_to_array
from tensorflow.keras.applications import VGG16, ResNet50, ResNet101V2
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, Flatten
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau
from sklearn.metrics.pairwise import cosine_similarity
from skimage.metrics import structural_similarity as ssim
import cv2

In [7]:
!unzip '/content/augmented_shape_data.zip'
!unzip '/content/generated_data_with_summation.zip'

Archive:  /content/augmented_shape_data.zip
   creating: augmented_shape_data/
   creating: augmented_shape_data/0/
  inflating: augmented_shape_data/0/Zero_1.png  
  inflating: augmented_shape_data/0/Zero_10.png  
  inflating: augmented_shape_data/0/Zero_100.png  
  inflating: augmented_shape_data/0/Zero_11.png  
  inflating: augmented_shape_data/0/Zero_12.png  
  inflating: augmented_shape_data/0/Zero_13.png  
  inflating: augmented_shape_data/0/Zero_14.png  
  inflating: augmented_shape_data/0/Zero_15.png  
  inflating: augmented_shape_data/0/Zero_16.png  
  inflating: augmented_shape_data/0/Zero_17.png  
  inflating: augmented_shape_data/0/Zero_18.png  
  inflating: augmented_shape_data/0/Zero_19.png  
  inflating: augmented_shape_data/0/Zero_2.png  
  inflating: augmented_shape_data/0/Zero_20.png  
  inflating: augmented_shape_data/0/Zero_21.png  
  inflating: augmented_shape_data/0/Zero_22.png  
  inflating: augmented_shape_data/0/Zero_23.png  
  inflating: augmented_shape_data/0

In [8]:
# Define dataset paths
dataset_path = '/content/augmented_shape_data'  # Path for single-shape images
combination_dataset_path = '/content/generated_data_with_summation'  # Path for combination-shape images

In [9]:
# Data Augmentation
train_datagen = ImageDataGenerator(
    rescale=1.0/255,
    validation_split=0.2,
    rotation_range=45,
    width_shift_range=0.5,
    height_shift_range=0.5,
    zoom_range=0.7,
    #horizontal_flip=True,
    fill_mode='nearest'
)

# Training and Validation Data Generators
train_generator = train_datagen.flow_from_directory(
    dataset_path, target_size=(224, 224), batch_size=32, class_mode='categorical', subset='training')

val_generator = train_datagen.flow_from_directory(
    dataset_path, target_size=(224, 224), batch_size=32, class_mode='categorical', subset='validation')

Found 800 images belonging to 10 classes.
Found 200 images belonging to 10 classes.


In [10]:

# Load Pretrained VGG16
base_model = ResNet101V2(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
x = Flatten()(base_model.output)
x = Dense(1024, activation='relu')(x)
x = Dense(512, activation='relu')(x)
x = Dense(128, activation='relu')(x)
output = Dense(train_generator.num_classes, activation='softmax')(x)

model = Model(inputs=base_model.input, outputs=output)

# Freeze Base Model Layers
for layer in base_model.layers:
    layer.trainable = True

# Compile Model
model.compile(optimizer=Adam(learning_rate=0.0001), loss='categorical_crossentropy', metrics=['accuracy'])

# Callbacks
checkpoint = ModelCheckpoint('/content/shape_classifier_best.keras', monitor='val_accuracy', save_best_only=True, mode='max', verbose=1)
early_stopping = EarlyStopping(monitor='val_loss', patience=5, verbose=1)
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=3, min_lr=0.00001, verbose=1)

# Train Model
history = model.fit(
    train_generator,
    validation_data=val_generator,
    epochs=20,
    callbacks = [checkpoint, early_stopping]
)

# Save Model
model.save('/content/shape_classifier.keras')

171317808/171317808 ━━━━━━━━━━━━━━━━━━━━ 1s 0us/step
Epoch 1/20


/usr/local/lib/python3.11/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:122: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 510ms/step - accuracy: 0.1798 - loss: 2.4985
Epoch 1: val_accuracy improved from -inf to 0.30000, saving model to /content/shape_classifier_best.keras
25/25 ━━━━━━━━━━━━━━━━━━━━ 183s 2s/step - accuracy: 0.1820 - loss: 2.4891 - val_accuracy: 0.3000 - val_loss: 2.3083
Epoch 2/20
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 552ms/step - accuracy: 0.4076 - loss: 1.6803
Epoch 2: val_accuracy improved from 0.30000 to 0.43000, saving model to /content/shape_classifier_best.keras
25/25 ━━━━━━━━━━━━━━━━━━━━ 73s 2s/step - accuracy: 0.4084 - loss: 1.6782 - val_accuracy: 0.4300 - val_loss: 1.6779
Epoch 3/20
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 555ms/step - accuracy: 0.4686 - loss: 1.5502
Epoch 3: val_accuracy improved from 0.43000 to 0.53500, saving model to /content/shape_classifier_best.keras
25/25 ━━━━━━━━━━━━━━━━━━━━ 46s 2s/step - accuracy: 0.4690 - loss: 1.5473 - val_accuracy: 0.5350 - val_loss: 1.6226
Epoch 4/20
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 552ms/step - accuracy: 0.5332 - loss: 1.30

In [11]:
# Load Model for Feature Extraction
feature_extractor = Model(inputs=model.input, outputs=model.layers[-2].output)

# Extract Features from Combination Dataset
features_combination = {}
for file in os.listdir(combination_dataset_path):
    img_path = os.path.join(combination_dataset_path, file)
    img = load_img(img_path, target_size=(224, 224))
    img_array = img_to_array(img) / 255.0
    img_array = np.expand_dims(img_array, axis=0)
    features = feature_extractor.predict(img_array).flatten()
    features_combination[file] = features

# Save Features
np.save('/content/features_combination.npy', features_combination)

1/1 ━━━━━━━━━━━━━━━━━━━━ 8s 8s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 54ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 43ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 41ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 36ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step
1/1 ━━━━━━━━━━

In [12]:
def match_test_image(test_image_path, features_combination, threshold=0.75, ssim_threshold=0.5):
    # Extract Features for Test Image
    img = load_img(test_image_path, target_size=(224, 224))
    img_array = img_to_array(img) / 255.0
    img_array = np.expand_dims(img_array, axis=0)
    test_features = feature_extractor.predict(img_array).flatten()

    # Compare with Combination Features
    best_match = None
    best_match_path = None
    best_score = 0
    best_ssim = 0

    for img_file, comb_features in features_combination.items():
        similarity = cosine_similarity([test_features], [comb_features])[0][0]

        if similarity > threshold:
            comb_img_path = os.path.join(combination_dataset_path, img_file)
            comb_img = cv2.imread(comb_img_path, cv2.IMREAD_COLOR)
            comb_img = cv2.resize(comb_img, (224, 224))

            test_img = cv2.imread(test_image_path, cv2.IMREAD_COLOR)
            test_img = cv2.resize(test_img, (224, 224))

            gray_test = cv2.cvtColor(test_img, cv2.COLOR_BGR2GRAY)
            gray_comb = cv2.cvtColor(comb_img, cv2.COLOR_BGR2GRAY)
            ssim_score = ssim(gray_test, gray_comb)

            if similarity > best_score :
                best_score = similarity
                best_match = img_file
                best_match_path = comb_img_path
                best_ssim = ssim_score

    if best_score >= threshold :
        return best_match_path, best_score, best_ssim
    else:
        return None, best_score, best_ssim


In [13]:
# Test Image Path
test_image_path = '/content/Five_Two_Zero_520_Noise.png'

# Match Test Image
matched_image_path, similarity_score, ssim_score = match_test_image(test_image_path, features_combination)

if matched_image_path:
    print(f"Match Found: {matched_image_path}")
    print(f"Cosine Similarity: {similarity_score}")
    print(f"SSIM Score: {ssim_score}")

    # Plot Test and Matched Images
    fig, axes = plt.subplots(1, 2, figsize=(12, 6))

    # Load Test Image
    test_img = load_img(test_image_path)
    axes[0].imshow(test_img)
    axes[0].set_title("Test Image")
    axes[0].axis('off')

    # Load Matched Image
    matched_img = load_img(matched_image_path)
    axes[1].imshow(matched_img)
    axes[1].set_title(f"Matched Image\n(Cosine: {similarity_score:.2f}, SSIM: {ssim_score:.2f})")
    axes[1].axis('off')

    plt.tight_layout()
    plt.show()
else:
    print("No Match Found!")
    plt.figure(figsize=(6, 6))
    test_img = load_img(test_image_path)
    plt.imshow(test_img)
    plt.title("Test Image (No Match Found)")
    plt.axis('off')
    plt.show()


FileNotFoundError: [Errno 2] No such file or directory: '/content/Five_Two_Zero_520_Noise.png'